In [2]:
## imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os
root = os.path.expanduser('~')

company_index='5'
company_name='Amazon'


In [3]:
def merge_data(company_list=[5], uni=False, bi=False, tri=False):
    sentences = []
    df_company_list = pd.read_csv( root + \
            '/Desktop/workspace/indeed/Job-Satisfaction/data/scraper_data/indeed_site50.csv')
    f50_list = pd.read_csv(root + \
            '/Desktop/workspace/indeed/Job-Satisfaction/data/Fortune_1000/fortune50_info.csv')
   
    data_companies = []
    for company_index in company_list:
        company_name = df_company_list.iloc[company_index - 1]['Company_Name']
        sector = f50_list.iloc[company_index - 1]['Sector']
        data_path_pros = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+\
                    f'{company_index}_{company_name}'+'/output_data/px_data4/pros.csv'
        data_path_cons = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+\
                    f'{company_index}_{company_name}'+'/output_data/px_data4/cons.csv'
        df_pros = pd.read_csv(data_path_pros)
        df_cons = pd.read_csv(data_path_cons)
        num_pros = df_pros.shape[0]
        num_cons = df_cons.shape[0]
        num_reviews = num_pros + num_cons
        avg_rating = df_pros.Ratings.mean()*.5 + df_cons.Ratings.mean()*.5
        if(company_name=='United-Parcel-Service-(UPS)'):
            company_name='UPS'
        data_companies.append([company_index, company_name, sector, num_pros, num_cons, num_reviews, avg_rating])
    
        
    return pd.DataFrame(data_companies, columns = ['Compnay_Index', 'Company_Name', 'Sector', 'Num_Pros', 'Num_Cons', 
                                           'Num_Reviews', 'Avg_Rating'])


df_companies = merge_data(company_list=range(1, 51))

In [4]:
df_companies.head()

,Compnay_Index,Company_Name,Sector,Num_Pros,Num_Cons,Num_Reviews,Avg_Rating
0,1,Walmart,Retailing,81133,78195,159328,3.447192
1,2,Exxonmobil,Energy,1204,1168,2372,3.886164
2,3,Apple,Technology,2863,2692,5555,4.052582
3,4,Berkshire-Hathaway,Financials,64,61,125,4.039447
4,5,Amazon,Retailing,14434,14250,28684,3.470209


In [5]:
df_companies.groupby('Sector')['Num_Reviews'].sum()

Sector
Aerospace & Defense            7396
Chemicals                      1216
Energy                         7879
Financials                    94941
Food & Drug Stores            28837
Food, Beverages & Tobacco      7946
Health Care                   49136
Household Products             2272
Industrials                    4127
Motor Vehicles & Parts         7226
Retailing                    315169
Technology                    27453
Telecommunications            72127
Transportation                49392
Name: Num_Reviews, dtype: int64

In [6]:
df_companies.groupby('Sector')['Company_Name'].count()

Sector
Aerospace & Defense          2
Chemicals                    1
Energy                       5
Financials                   9
Food & Drug Stores           2
Food, Beverages & Tobacco    2
Health Care                  7
Household Products           1
Industrials                  1
Motor Vehicles & Parts       2
Retailing                    7
Technology                   6
Telecommunications           3
Transportation               2
Name: Company_Name, dtype: int64

# Make sectorwise data

In [7]:
## making dataset

def get_px_data(company_index=5, company_name='Amazon',
                uni=True, bi=True, tri=True, procon = 'pros'):
    if procon == 'pros':
        data_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+\
                    f'{company_index}_{company_name}'+'/output_data/px_data4/pros.csv'
    elif procon == 'cons':
        data_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+\
                    f'{company_index}_{company_name}'+'/output_data/px_data4/cons.csv'
    else: 
        data_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+\
                    f'{company_index}_{company_name}'+'/output_data/px_data_reviews1/reviews.csv'
    df = pd.read_csv(data_path)
#     print(df.head())
    df_data = pd.DataFrame()
#     df_data['Company_Index'] = company_index
    
    df_data['Ratings'] = df['Ratings']
    df_data['Job_Status'] = df['Job_Status']
    df_data['Reviewed_Year'] = df['Reviewed_Year'].astype("int")
    df_data['Review_Text'] = df['Review_Text']
    
    
    if uni and bi and tri:
        df_data['Px_Texts'] = df['Unigrams'].fillna('') + ' ' +\
        df['Bigrams'].fillna('') + ' ' + df['Trigrams'].fillna('')
    elif uni:
        df_data['Px_Texts'] = df['Unigrams'].dropna()
    elif bi:
        df_data['Px_Texts'] = df['Bigrams'].dropna()
    elif tri:
        df_data['Px_Texts'] = df['Trigrams'].dropna()
    df_data['Tknz_Texts'] = df_data['Px_Texts'].str.split()
    df_data['Doc_Length'] = df_data['Tknz_Texts'].str.len()
    
    df_data['Work_Life'] = df['Work_Life']
    df_data['Benefits'] = df['Benefits']
    df_data['Job_Advancement'] = df['Job_Advancement']
    df_data['Management'] = df['Management']
    df_data['Culture'] = df['Culture']
#     df_data['Pro_Con'] = procon
    df_data['Company_Index'] = company_index
    df_data['Company_Name'] = (company_name)
#     df_data = df_data[df_data.Doc_Length > 3]
    return df_data

# display(get_px_data().head())
# display(get_px_data().shape[0]+get_px_data(procon='cons').shape[0])
get_px_data(procon='pros').head()

,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name
0,5.0,0,2019,"Free drinks, paid time, overtime",drink free overtim paid time free_drink paid_t...,"[drink, free, overtim, paid, time, free_drink,...",8,5.0,5.0,3.0,5.0,5.0,5,Amazon
1,5.0,0,2019,"On your own, flexible, can keep job even worki...",flexibl month time flexibl_time,"[flexibl, month, time, flexibl_time]",4,4.0,2.0,1.0,1.0,3.0,5,Amazon
2,1.0,0,2019,Great benefits,benefit,[benefit],1,2.0,5.0,1.0,1.0,1.0,5,Amazon
3,2.0,0,2019,Safety,safeti,[safeti],1,2.0,4.0,1.0,1.0,1.0,5,Amazon
4,4.0,0,2019,Nice break room,break nice nice_break,"[break, nice, nice_break]",3,3.0,4.0,5.0,5.0,5.0,5,Amazon


In [9]:
df_companies.groupby('Sector')['Num_Reviews'].sum()

Sector
Aerospace & Defense            7396
Chemicals                      1216
Energy                         7879
Financials                    94941
Food & Drug Stores            28837
Food, Beverages & Tobacco      7946
Health Care                   49136
Household Products             2272
Industrials                    4127
Motor Vehicles & Parts         7226
Retailing                    315169
Technology                    27453
Telecommunications            72127
Transportation                49392
Name: Num_Reviews, dtype: int64

In [19]:
f50_list = pd.read_csv(root + \
            '/Desktop/workspace/indeed/Job-Satisfaction/data/Fortune_1000/fortune50_info.csv')

# def merge_data(company_list=[5], uni=False, bi=False, tri=False):
#     sentences = []
#     df_company_list = pd.read_csv( root + \
#             '/Desktop/workspace/indeed/Job-Satisfaction/data/scraper_data/indeed_site50.csv')
#     sentences = pd.DataFrame(columns = ['Ratings', 'Job_Status', 'Reviewed_Year', 'Review_Text', 'Px_Texts',
#                                         'Tknz_Texts', 'Doc_Length', 'Work_Life', 'Benefits', 'Job_Advancement', 
#                                         'Management', 'Culture', 'Company_Index', 'Company_Name'])
#     for company_index in company_list:
#         company_name = df_company_list.iloc[company_index - 1]['Company_Name']
#         px_sents = get_px_data(company_index=company_index, company_name=company_name, 
#                         uni=uni, bi=bi, tri=tri, procon='cons')
# #         if (px_sents.shape[0]) < 1000:
#         sentences = pd.concat([sentences, px_sents], ignore_index=True)
# #         else:
# # #             sentences = pd.concat([sentences, px_sents.sample(n=1000, random_state=42)],\
# # #                                   ignore_index=True)
# #             sentences = pd.concat([sentences, px_sents.sample(n=1000, random_state=6)],\
# #                                   ignore_index=True)
#     return sentences

def merge_data(company_list=[5], uni=False, bi=False, tri=False):
    sentences = []
    df_company_list = pd.read_csv( root + \
            '/Desktop/workspace/indeed/Job-Satisfaction/data/scraper_data/indeed_site50.csv')
    sentences = pd.DataFrame(columns = ['Ratings', 'Job_Status', 'Reviewed_Year', 'Review_Text', 'Px_Texts',
                                        'Tknz_Texts', 'Doc_Length', 'Work_Life', 'Benefits', 'Job_Advancement', 
                                        'Management', 'Culture', 'Company_Index', 'Company_Name'])
    for company_index in company_list:
        company_name = df_company_list.iloc[company_index - 1]['Company_Name']
        px_sents = get_px_data(company_index=company_index, company_name=company_name, 
                        uni=uni, bi=bi, tri=tri, procon='pros')
        if (px_sents.shape[0]) < 5000:
            sentences = pd.concat([sentences, px_sents], ignore_index=True)
        else:
#             sentences = pd.concat([sentences, px_sents.sample(n=1000, random_state=42)],\
#                                   ignore_index=True)
            sentences = pd.concat([sentences, px_sents.sample(n=5000, random_state=6)],\
                                  ignore_index=True)
    return sentences

# sentences = merge_data(company_list=[5],
#                         uni=True, bi=True, tri=True)
companies = []
for company_index in range(1, 51):
    sector = f50_list.iloc[company_index - 1]['Sector']
    if(sector == 'Retailing'):
        companies.append(company_index)
        
sentences = merge_data(company_list=companies,
                        uni=True, bi=True, tri=True)
# sentences = merge_data(company_list=[5, 6, 7, 8, 9, 10],
#                         uni=True, bi=True, tri=True)
# print(len(sentences))
display(sentences.head())
display(sentences.shape)

path = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/sector/Retailing/pro'
if not os.path.exists(path):
    os.makedirs(path)
sentences.to_csv(path+'px.csv')

,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name
0,4.0,0,2018,"Discount, health benefits",benefit discount health discount_health health...,"[benefit, discount, health, discount_health, h...",5,4.0,4.0,4.0,4.0,4.0,1,Walmart
1,1.0,0,2019,1 hr and 30 minute break combined,break hr minut minut_break,"[break, hr, minut, minut_break]",4,1.0,1.0,1.0,1.0,1.0,1,Walmart
2,3.0,1,2013,flexible hours,flexibl hour flexibl_hour,"[flexibl, hour, flexibl_hour]",3,3.0,2.0,2.0,3.0,2.0,1,Walmart
3,5.0,0,2019,"Discounts, Hour lunches",discount hour lunch discount_hour hour_lunch d...,"[discount, hour, lunch, discount_hour, hour_lu...",6,5.0,5.0,2.0,4.0,5.0,1,Walmart
4,4.0,0,2016,"Hour lunch, decent pay, good co-workers",cowork decent hour lunch pay decent_pay hour_l...,"[cowork, decent, hour, lunch, pay, decent_pay,...",11,4.0,2.0,3.0,1.0,4.0,1,Walmart


(29773, 14)